In [ ]:
import cv2
import numpy as np
image_1 = "small.png"           # your goal image
image_0 = "square1.png"         # your initial region
out = "white.png"               # output file

# all these photos should be the same dimensions, and be square


def main():
    my_image_0 = cv2.imread(image_0)            # import original region image
    my_image_1 = cv2.imread(image_1)            # import destination region image
    out_image = cv2.imread(out)                 # import image to print on
    # my_image_0 = noisify(my_image_0)            # add noise to original region
    region_array = region_definition(my_image_0)    # define array u based on original region image
    print_photo(region_array, out_image, 0)
    f_matrix = functional(my_image_1)     # calculate functional values
    i = 0
    delta_t = 1*10**(-5)
    while i < 2500:                             # iterate through algorithm
        (array_out, delta_t) = level_curves_algorithm(region_array, f_matrix, delta_t)
        i += 1
        print(delta_t)
        region_array = array_out.copy()
        print(i)
        if i % 10 == 0:            # print region every n steps, making n larger will make algorithm run slightly faster
            print_photo(region_array, out_image, i)
    print_photo(region_array, out_image, i)


def region_definition(image):                   # function to define the array u based on original region
    # the definition of this region is currently specified to a black and white image. this will change when we move to
    # images of cells and define criteria used to identify region. We can use the function Stefan wrote here to define
    # the input region based on a drawn circle.
    cellpixels = []
    outerpixels = []
    (height, width, x) = image.shape            # define array based on image shape + 2 for width and height
    array = [[-1 for i in range(height + 2)] for j in range(width + 2)]         # default values to -1
    for x in range(0, width):                   # iterate through image and check pixel values
        for y in range(0, height):
            if image[x, y][0] == 0 and image[x, y][1] == 0 and image[x, y][2] == 255:
                cellpixels.append((x,y))
                array[y + 1][x + 1] = 1
            else:
                outerpixels.append((x,y))          # if pixel value is 0 (black) pixel is in region
    array[0][:] = array[1][:]                   # pad outer rows and columns of array
    array[height + 1][:] = array[height][:]
    array[:][0] = array[:][1]
    array[:][width + 1] = array[:][width]
    return array

def average_pixels():
    imgfile = r"C:\Users\stefa\Desktop\Queens\Fourth Year\MTHE493\Frames\scene00001.png" # original picuture
    cellsum = 0
    outsum = 0
    img = cv2.imread(imgfile)
    grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    for i in cellpixels:
        cellsum = cellsum + grayimg[i[0], i[1]]
    for i in outerpixels:
        outsum = outsum + grayimg[i[0], i[1]]
    cellavg = cellsum/len(cellpixels)
    outavg = outsum/len(outerpixels)
    return (cellavg, outavg)
    
def noisify(photo):                             # function to add noise to initial image
    (height, width, x) = photo.shape
    mean = 0                              # defining gaussian white noise, mean of zero, variance can change as desired
    var = 30
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, (height, width))
    for x in range(0, height):            # iterate through image array and add noise
        for y in range(0, width):         # if output values are outside of [0, 255], set it to the max or the min
            if photo[y][x][0] + gaussian[x, y] < 0:
                photo[y][x][0] = 0
            elif photo[y][x][0] + gaussian[x, y] > 255:
                photo[y][x][0] = 255
            else:
                photo[y][x][0] = photo[y][x][0] + gaussian[x, y]
    return photo


def functional(my_image):  # function to define the functional of our region. Currently based on Mumford-Shaw functional
    a = 0                       # but this will change as we define invariant properties of our region
    b = 255
    (height, width, x) = my_image.shape
    F = [[0 for i in range(height)] for j in range(width)]
    for x in range(0, width):
        for y in range(0, height):
            F[y][x] = (int(my_image[y][x][0]) - ((a+b)/2))*(b - a)*2
    return F


def level_curves_algorithm(array1, f_matrix, delta_t):     # function to implement level curves algorithm as defined in
    delta_t_new = 0                                        # Mansouris paper
    epsilon = 10**(-100)
    lambd = 10
    height = len(array1[0])
    width = len(array1)
    array2 = [[-1 for i in range(height)] for j in range(width)]
    for x in range(1, width - 1):
        for y in range(1, height - 1):
            x1 = array1[y][x + 1]
            x2 = array1[y][x - 1]
            y1 = array1[y + 1][x]
            y2 = array1[y - 1][x]
            xy = array1[y][x]
            ux = (x1 - x2)/2
            uy = (y1 - y2)/2
            uxx = x1 - 2*xy + x2
            uyy = y1 - 2*xy + y2
            uxy = (array1[y + 1][x + 1] - array1[y + 1][x - 1] - array1[y - 1][x + 1] + array1[y - 1][x - 1])/4
            diff = (ux**2) + (uy**2)
            dpx = x1 - xy
            dmx = xy - x2
            dpy = y1 - xy
            dmy = xy - y2
            poss = .5/(abs(ux) + abs(uy) + epsilon)
            if 0.7 > poss > delta_t_new and (xy*x1 < 0 or xy*x2 < 0 or xy*y1 < 0 or xy*y2 < 0):
                delta_t_new = poss  # defining time-step based off Courant–Friedrichs–Lewy condition
            gradient_plus = (max(dmx, 0)**2 + min(dpx, 0)**2 + max(dmy, 0)**2 + min(dpy, 0)**2)**1/2
            gradient_minus = (max(dpx, 0)**2 + min(dmx, 0)**2 + max(dpy, 0)**2 + min(dmy, 0)**2)**1/2
            k = (uxx*(uy**2) - 2*uy*ux*uxy + uyy*(ux**2))/((diff + epsilon)**(3/2))
            new = xy + delta_t*(-(max(f_matrix[y - 1][x - 1], 0)*gradient_plus + min(f_matrix[y - 1][x - 1], 0)*gradient_minus) + lambd*k*(diff**(1/2)))
            if new > 1000000000000000000000000:
                new = 1000000000000000000000000
            elif new < -1000000000000000000000000:
                new = - 1000000000000000000000000
            array2[y][x] = new
    return array2, delta_t_new


def print_photo(array, image, i):           # print out resulting region, will need to update as we input functional
    height = len(array[0])
    width = len(array)
    for x in range(1, width - 1):
        for y in range(1, height - 1):
            if array[y][x] < 0:
                image[y - 1, x - 1][0] = 255
                image[y - 1, x - 1][1] = 255
                image[y - 1, x - 1][2] = 255
            elif array[y][x] > 0:
                image[y - 1, x - 1][0] = 0
                image[y - 1, x - 1][1] = 0
                image[y - 1, x - 1][2] = 0
        if i < 10:
            cv2.imwrite("image000" + str(i) + ".png", image)
        elif i < 100:
            cv2.imwrite("image00" + str(i) + ".png", image)
        elif i < 1000:
            cv2.imwrite("image0" + str(i) + ".png", image)
        else:
            cv2.imwrite("image" + str(i) + ".png", image)